Read in all data

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error
from utils import stratified_sample, annotate, even_train_split
import glob  # Importing the glob module to find all the files matching a pattern

# Pattern to match the data files
file_pattern = "data_files/user_*/metric_df.csv"

# Initialize a dictionary to store Dataframes for each dataset
all_datasets = {}

# Loop through each file that matches the file pattern
for filepath in glob.glob(file_pattern):
    # print(filepath)
    # print(filepath.split('/'))
    # user_name = filepath.split('/')[1]
    user_name = filepath.split('\\')[1]
    print(f"Processing {filepath} dataset...")

    # Read in data file as a pandas dataframe
    data = pd.read_csv(filepath, index_col=0)

    # add weighted performance metric
    w = 1
    data["weighted_performance"] = 10*data['throughput'] - w*(data['avg_osd'] + data['avg_target_error'])

    all_datasets[user_name] = data

# Combine datasets for Lizzie
lizzie1 = all_datasets["user_lizzie1"]
lizzie2 = all_datasets["user_lizzie2"]
combined_df = pd.concat([lizzie1, lizzie2])
all_datasets["user_lizzie"] = combined_df.groupby(['latency', 'scale']).mean().reset_index()

Modeling

In [14]:
from models import PolyRegression, GPRegression
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel, RationalQuadratic, WhiteKernel
import utils
import json
import warnings
import logging

## Choose model to apply, returning predictions over original dataset and dense inputs
# model_type = "Poly2"
# model_type = "GPR_RBF_default"
# model_type = "GPR_RBF_anisotropic"
# model_type = "GPR_RBF_Noise_default"
# model_type = "GPR_RBF_Noise_anisotropic"
# model_type = "GPR_RQ_default"
model_type = "GPR_RQ_Noise_default"
# model_type = "blah"

# Configure logging to write to a file
logging.basicConfig(filename='warnings_log.txt', level=logging.WARNING, format='%(message)s')

# Function to redirect warnings to logging
def warn_to_logging(message, category, filename, lineno, file=None, line=None):
    logging.warning(f'{filename}:{lineno}: {category.__name__}: {message}')

# Redirect all warnings to the warn_to_logging function
warnings.showwarning = warn_to_logging

all_results = {}
output_metrics = ["throughput", "avg_target_error", "avg_osd", "avg_movement_speed", "weighted_performance"]
for output_metric in output_metrics:
	
	print(output_metric)
	user_results = {}
	for user, data in list(all_datasets.items()): 
		# if user == "user_lizzie" or user == "user_lizzie1":
		# 	continue
		print(f"\t{user}")

		# Prepare data 
		X = data[['latency', 'scale']]
		Y = data[output_metric]

		# Initialize evaluation metrics
		optimal_match_rate = []
		optimal_scale_error = []
		mse_scores = []
		full_mse_scores = []
		n_train_mse = []
		n_train_full_mse = []
		n_train_p = []

		n = len(data)
		n_train_values = range(2, n-1)
		for n_train in n_train_values:

			n_train_p.append(n_train / n)
			# Split into training/test sets
			# X_train, X_test, Y_train, Y_test = train_test_split(X, Y, train_size=n_train/n)
			train_set, test_set = even_train_split(data, n_train)
			X_train, X_test = train_set[['latency', 'scale']], test_set[['latency', 'scale']]
			Y_train, Y_test = train_set[output_metric], test_set[output_metric]
			
			# Create dense test input
			# latency_set = data['latency'].unique()# np.arange(0.0, 0.76, 0.01)
			# latency_range = np.array(data['latency'].unique()) #np.linspace(latency_set.min(), latency_set.max(), 50)
			latency_range = np.arange(0.0, data['latency'].max()+0.01, 0.01)
			scale_range = np.arange(data['scale'].min(), data['scale'].max()+0.025, 0.025) #np.linspace(data['scale'].min(), data['scale'].max(), 50)
			latency_grid, scale_grid = np.meshgrid(latency_range, scale_range)
			X_dense = np.c_[latency_grid.ravel(), scale_grid.ravel()]
			X_dense = np.round(X_dense, 3)
			
			# # Polynomial Regression
			if model_type.startswith("Poly"):
				degree = int(model_type.strip("Poly"))
				Y_pred, model_params = PolyRegression(X_train.values, Y_train.values, X.values, degree)
				Y_pred_dense, _ = PolyRegression(X_train.values, Y_train.values, X_dense, degree)

			# Gaussian Process Regression
			elif model_type.startswith("GPR"):
				# Choose kernel
				kernel_type = model_type.removeprefix("GPR_")
				# print(kernel_type)
				if kernel_type == "RBF_Noise_default":
					ConstantKernel() * RBF() + WhiteKernel() # Default RBF with likelihood noise
				elif kernel_type == "RBF_anisotropic":
					kernel = ConstantKernel() * RBF([1.0, 1.0])
				elif kernel_type == "RBF_Noise_anisotropic":
					kernel = ConstantKernel() * RBF([1.0, 1.0]) + WhiteKernel() # RBF with anistropic length scale
				elif kernel_type == "RQ_Noise_default":
					kernel = ConstantKernel() * RationalQuadratic() + WhiteKernel() # Default Rational Quadratic with likelihood noise
				else:
					print("Invalid kernel specification!")
					break
				
				Y_pred, Y_pred_std, model_params = GPRegression(X_train.values, Y_train.values, X.values, kernel)
				Y_pred_dense, Y_pred_std, _ = GPRegression(X_train.values, Y_train.values, X_dense, kernel)

			else:
				print("Invalid model type specification!")
				break

			## Evaluate metrics
			dense_df = pd.DataFrame({
					'latency': X_dense[:, 0].flatten(),
					'scale': X_dense[:, 1].flatten(),
					'Y_pred_dense': Y_pred_dense.flatten()
				})
			data["Y_pred"] = Y_pred

			# Mean Square Error on whole dataset
			full_mse = mean_squared_error(Y, Y_pred)
			if True: #full_mse < 5000:
				n_train_full_mse.append(n_train)
				full_mse_scores.append(full_mse)

			# Mean Square Error on test set
			Y_test_pred = data.loc[Y_test.index]["Y_pred"]
			mse = mean_squared_error(Y_test, Y_test_pred)
			if True: #mse < 5000:
				n_train_mse.append(n_train)
				mse_scores.append(mse)
			
			if output_metric in ["throughput", "avg_movement_speed", "weighted_performance"]: # optimal scale at maximum
				optimal_scale_dense = dense_df.loc[dense_df.groupby('latency')['Y_pred_dense'].idxmax()][['latency', 'scale']]
				optimal_scale_ref = data.loc[data.groupby('latency')[output_metric].idxmax()][['latency', 'scale']]
				optimal_scale_pred = data.loc[data.groupby('latency')['Y_pred'].idxmax()][['latency', 'scale']]
			else: # optimal scale at minimum
				optimal_scale_dense = dense_df.loc[dense_df.groupby('latency')['Y_pred_dense'].idxmin()][['latency', 'scale']]
				optimal_scale_ref = data.loc[data.groupby('latency')[output_metric].idxmin()][['latency', 'scale']]
				optimal_scale_pred = data.loc[data.groupby('latency')['Y_pred'].idxmin()][['latency', 'scale']]

			# Merge the results on 'latency'
			merged_ref_pred = pd.merge(optimal_scale_ref, optimal_scale_pred, 
								on='latency', suffixes=('_ref', '_pred'))
			
			merged_ref_dense = pd.merge(optimal_scale_ref, optimal_scale_dense, 
								on='latency', suffixes=('_ref', '_dense'))
			# print(optimal_scale_dense)
			# print(merged_ref_dense)
			

			# Count the number of matches
			matches = (merged_ref_pred['scale_ref'] == merged_ref_pred['scale_pred']).sum()
			scale_error = np.abs(merged_ref_dense['scale_ref'] - merged_ref_dense['scale_dense']).mean()

			optimal_match_rate.append(matches / len(optimal_scale_ref))
			optimal_scale_error.append(scale_error)

			# Visualize model prediction
			if n_train == n-2:
				utils.model_heatmaps(data, dense_df, X_train, user, output_metric, model_type, model_params)

		else:
			# Store results from this dataset
			user_results[user] = {
				'n_train_mse': list(n_train_mse),
				'n_train_full_mse': list(n_train_full_mse),
				'full_mse_scores': full_mse_scores,
				'mse_scores': mse_scores,
				'n_train_all': list(n_train_values),
				'match_rate': optimal_match_rate,
				'scale_error': optimal_scale_error,
				'n_train_p': n_train_p
			}
			continue
		break	
	else:
		all_results[output_metric] = user_results
		continue
	break

with open(f"model_result_data/{model_type}.json", "w") as file:
	json.dump(all_results, file)



throughput
	user_jason
	user_lauren
	user_lizzie1
	user_lizzie2
	user_sarah1
	user_shreya
	user_sujaan
	user_xiao
	user_yutong
	user_lizzie
avg_target_error
	user_jason
	user_lauren
	user_lizzie1
	user_lizzie2
	user_sarah1
	user_shreya
	user_sujaan
	user_xiao
	user_yutong
	user_lizzie
avg_osd
	user_jason
	user_lauren
	user_lizzie1
	user_lizzie2
	user_sarah1
	user_shreya
	user_sujaan
	user_xiao
	user_yutong
	user_lizzie
avg_movement_speed
	user_jason
	user_lauren
	user_lizzie1
	user_lizzie2
	user_sarah1
	user_shreya
	user_sujaan
	user_xiao
	user_yutong
	user_lizzie
weighted_performance
	user_jason
	user_lauren
	user_lizzie1
	user_lizzie2
	user_sarah1
	user_shreya
	user_sujaan
	user_xiao
	user_yutong
	user_lizzie


Visualize

In [ ]:
# # Plotting the results for all datasets
import matplotlib.pyplot as plt

# Load data
## Choose model to apply, returning predictions over original dataset and dense inputs
# model_type = "Poly2"
# model_type = "GPR_RBF_default"
# model_type = "GPR_RBF_anisotropic"
# model_type = "GPR_RBF_Noise_default"
model_type = "GPR_RBF_Noise_anisotropic"
# model_type = "GPR_RQ_default"
# model_type = "GPR_RQ_Noise_defualt"
# model_type = "blah"

with open(f"model_result_data/{model_type}.json", "r") as file:
	all_results = json.load(file)

for output_metric, user_results in all_results.items():

	fig, axes = plt.subplots(2, 2, figsize=(16, 8))
	fig.suptitle(f"Model Evaluation Metrics for {model_type} predicting {output_metric}")
	for user, results in user_results.items():
		axes[0, 0].plot(results['n_train_p'], results['match_rate'], marker='o', label=user)
		axes[0, 1].plot(results['n_train_p'], results['scale_error'], marker='o', label=user)
		axes[1, 0].plot(results['n_train_p'], results['full_mse_scores'], marker='o', label=user)
		axes[1, 1].plot(results['n_train_p'], results['mse_scores'], marker='o', label=user)

	axes[0, 0].set_title("Optimal Scale Prediction Rate")
	axes[0, 0].set_xlabel("Training Set Proportion")
	axes[0, 0].set_ylabel("Percentage of Correct Predictions")

	axes[0, 1].set_title("Optimal Scale Prediction Error Using Dense Prediction")
	axes[0, 1].set_xlabel("Training Set Proportion")
	axes[0, 1].set_ylabel("Avg Error")


	axes[1, 0].set_title('MSE on whole dataset')
	axes[1, 0].set_xlabel('Training Set Proportion')
	axes[1, 0].set_ylabel('Model Accuracy (MSE Score)')

	axes[1, 1].set_title('MSE on test set')
	axes[1, 1].set_xlabel('Training Set Proportion')
	axes[1, 1].set_ylabel('Model Accuracy (MSE Score)')
	axes[1, 0].legend()

	plt.tight_layout()
	plt.savefig(f"figures/model_results/{output_metric}/{model_type}_model_eval_metrics_{output_metric}.png", facecolor='w')
	plt.show()


In [ ]:
## Plot averaged results

for output_metric, user_results in all_results.items():
	fig, axes = plt.subplots(2, 2, figsize=(16, 8))
	fig.suptitle(f"Model Evaluation Metrics for {model_type} predicting {output_metric}, mean over users")

	# Prepare lists of lists to store data
	match_rate_lists = []
	scale_error_lists = []
	full_mse_score_lists = []
	mse_score_lists = []
	n_train_lists = []

	# Collect data for each metric
	for user, results in user_results.items():
		match_rate_lists.append(results['match_rate'])
		scale_error_lists.append(results['scale_error'])
		full_mse_score_lists.append(results['full_mse_scores'])
		mse_score_lists.append(results['mse_scores'])
		n_train_lists.append(results['n_train_all'])

	# Function to calculate average and standard deviation safely
	def safe_mean_std(data_lists, index):
		valid_data = [data[index] for data in data_lists if index < len(data)]
		return np.mean(valid_data), np.std(valid_data)

	# Calculate averages and standard deviations safely
	avg_match_rate = []
	std_match_rate = []
	avg_scale_error = []
	std_scale_error = []
	avg_full_mse_scores = []
	std_full_mse_scores = []
	avg_mse_scores = []
	std_mse_scores = []

	min_n_train_length = min([len(data_list) for data_list in n_train_lists])
	min_n_train_list = n_train_lists[0][:min_n_train_length]
	for n in range(min_n_train_length):
		mean, std = safe_mean_std(match_rate_lists, n)
		avg_match_rate.append(mean)
		std_match_rate.append(std)

		mean, std = safe_mean_std(scale_error_lists, n)
		avg_scale_error.append(mean)
		std_scale_error.append(std)

		mean, std = safe_mean_std(full_mse_score_lists, n)
		avg_full_mse_scores.append(mean)
		std_full_mse_scores.append(std)

		mean, std = safe_mean_std(mse_score_lists, n)
		avg_mse_scores.append(mean)
		std_mse_scores.append(std)

	# Plotting the average values and standard deviation
	# Plotting the average values and standard deviation
	axes[0, 0].plot(min_n_train_list, avg_match_rate, marker='o', label='Mean over users')
	axes[0, 0].fill_between(min_n_train_list, np.subtract(avg_match_rate, std_match_rate), 
							np.add(avg_match_rate, std_match_rate), alpha=0.2)

	axes[0, 1].plot(min_n_train_list, avg_scale_error, marker='o', label='Mean over users')
	axes[0, 1].fill_between(min_n_train_list, np.subtract(avg_scale_error, std_scale_error), 
							np.add(avg_scale_error, std_scale_error), alpha=0.2)

	axes[1, 0].plot(min_n_train_list, avg_full_mse_scores, marker='o', label='Mean over users')
	axes[1, 0].fill_between(min_n_train_list, np.subtract(avg_full_mse_scores, std_full_mse_scores), 
							np.add(avg_full_mse_scores, std_full_mse_scores), alpha=0.2)

	axes[1, 1].plot(min_n_train_list, avg_mse_scores, marker='o', label='Mean Over Users')
	axes[1, 1].fill_between(min_n_train_list, np.subtract(avg_mse_scores, std_mse_scores), 
							np.add(avg_mse_scores, std_mse_scores), alpha=0.2)
	
	axes[0, 0].set_title("Optimal Scale Prediction Rate")
	axes[0, 0].set_xlabel("Training Set Proportion")
	axes[0, 0].set_ylabel("Percentage of Correct Predictions")

	axes[0, 1].set_title("Optimal Scale Prediction Error Using Dense Prediction")
	axes[0, 1].set_xlabel("Training Set Proportion")
	axes[0, 1].set_ylabel("Avg Error")


	axes[1, 0].set_title('MSE on whole dataset')
	axes[1, 0].set_xlabel('Training Set Proportion')
	axes[1, 0].set_ylabel('Model Accuracy (MSE Score)')

	axes[1, 1].set_title('MSE on test set')
	axes[1, 1].set_xlabel('Training Set Proportion')
	axes[1, 1].set_ylabel('Model Accuracy (MSE Score)')
	# axes[1, 0].legend()

	plt.tight_layout()
	plt.savefig(f"figures/model_results/{output_metric}/{model_type}_model_eval_metrics_{output_metric}_average.png", facecolor='w')
	plt.show()

In [ ]:
## Plot optimal scale

output_metrics = ["throughput", "avg_target_error", "avg_osd", "avg_movement_speed", "weighted_performance"]
for output_metric in output_metrics:
	# print(output_metric)
	user_results = {}
	plt.figure(figsize=(12, 6))
	for user, data in list(all_datasets.items()):

		if user == "user_lizzie" or user == "user_lizzie1":
			continue

		X = data[['latency', 'scale']]
		latency_range = np.arange(0.0, data['latency'].max()+0.01, 0.01)
		scale_range = np.linspace(data['scale'].min(), data['scale'].max(), 50)
		latency_grid, scale_grid = np.meshgrid(latency_range, scale_range)
		X_dense = np.c_[latency_grid.ravel(), scale_grid.ravel()]
		X_dense = np.round(X_dense, 3)

		## Choose model to apply, training on whole dataset, returning predictions over dense input
		# model_type = "GPR_RBF_default"
		model_type = "GPR_RQ_default"
		# model_type = "Poly2"
		
		# # Polynomial Regression
		# degree = 2
		# Y_pred_dense = PolyRegression(X.values, Y.values, X_dense, degree)

		# Gaussian Process Regression
		# kernel = ConstantKernel() * RBF() # Default RBF
		kernel = ConstantKernel() * RationalQuadratic() # Default Rational Quadratic
		Y_pred_dense, Y_pred_std = GPRegression(X.values, Y.values, X_dense, kernel)

		dense_df = pd.DataFrame({
					'latency': X_dense[:, 0].flatten(),
					'scale': X_dense[:, 1].flatten(),
					'Y_pred_dense': Y_pred_dense.flatten()
				})		

		if output_metric in ["throughput", "avg_movement_speed", "weighted_performance"]: # optimal scale at maximum
			optimal_scale_dense = dense_df.loc[dense_df.groupby('latency')['Y_pred_dense'].idxmax()][['latency', 'scale']]
			optimal_scale_ref = data.loc[data.groupby('latency')[output_metric].idxmax()][['latency', 'scale']]
		else: # optimal scale at minimum
			optimal_scale_dense = dense_df.loc[dense_df.groupby('latency')['Y_pred_dense'].idxmin()][['latency', 'scale']]
			optimal_scale_ref = data.loc[data.groupby('latency')[output_metric].idxmin()][['latency', 'scale']]

		# print(optimal_scale_ref)
		plt.title(f"Optimal Scale by {output_metric}")
		plt.xlabel("latency")
		plt.ylabel("scaling factor")
		plt.plot(optimal_scale_ref['latency'], optimal_scale_ref['scale'], marker='o', label=user)
	plt.legend()
	plt.savefig(f"figures/optimal_scale_per_latency/{output_metric}.png")
	plt.show()
		

In [ ]:
## Plot optimal scale per latency with model prediction

output_metrics = ["throughput", "avg_target_error", "avg_osd", "avg_movement_speed", "weighted_performance"]
for output_metric in output_metrics:
	# print(output_metric)
	user_results = {}
	plt.figure(figsize=(12, 6))
	for user, data in list(all_datasets.items())[:1]:

		if user == "user_lizzie" or user == "user_lizzie1":
			continue

		X = data[['latency', 'scale']]
		latency_range = np.arange(0.0, data['latency'].max()+0.01, 0.01)
		scale_range = np.linspace(data['scale'].min(), data['scale'].max(), 50)
		latency_grid, scale_grid = np.meshgrid(latency_range, scale_range)
		X_dense = np.c_[latency_grid.ravel(), scale_grid.ravel()]
		X_dense = np.round(X_dense, 3)

		## Choose model to apply, training on whole dataset, returning predictions over dense input
		# model_type = "GPR_RBF_default"
		model_type = "GPR_RQ_default"
		# model_type = "Poly2"
		
		# # Polynomial Regression
		# degree = 2
		# Y_pred_dense = PolyRegression(X.values, Y.values, X_dense, degree)

		# Gaussian Process Regression
		# kernel = ConstantKernel() * RBF() # Default RBF
		kernel = ConstantKernel() * RationalQuadratic() # Default Rational Quadratic
		Y_pred_dense, Y_pred_std = GPRegression(X.values, Y.values, X_dense, kernel)

		dense_df = pd.DataFrame({
					'latency': X_dense[:, 0].flatten(),
					'scale': X_dense[:, 1].flatten(),
					'Y_pred_dense': Y_pred_dense.flatten()
				})		

		if output_metric in ["throughput", "avg_movement_speed", "weighted_performance"]: # optimal scale at maximum
			optimal_scale_dense = dense_df.loc[dense_df.groupby('latency')['Y_pred_dense'].idxmax()][['latency', 'scale']]
			optimal_scale_ref = data.loc[data.groupby('latency')[output_metric].idxmax()][['latency', 'scale']]
		else: # optimal scale at minimum
			optimal_scale_dense = dense_df.loc[dense_df.groupby('latency')['Y_pred_dense'].idxmin()][['latency', 'scale']]
			optimal_scale_ref = data.loc[data.groupby('latency')[output_metric].idxmin()][['latency', 'scale']]

		# print(optimal_scale_ref)
		plt.title(f"Optimal Scale by {output_metric}")
		plt.xlabel("latency")
		plt.ylabel("scaling factor")
		plt.scatter(optimal_scale_ref['latency'], optimal_scale_ref['scale'], marker='x', label="measured")
		plt.plot(optimal_scale_dense['latency'], optimal_scale_dense['scale'], label="predicted")
	plt.legend()
	# plt.savefig(f"figures/optimal_scale_per_latency/{output_metric}.png")
	plt.show()

In [ ]:
## Plot key metric heatmaps
from utils import annotate_extrema

metric_df = all_datasets["user_lizzie"]
# Create a 2x5 subplot for the heatmaps
fig, axes = plt.subplots(1, 3, figsize=(16, 6))

# Define a function to highlight the maximum value in each row
def highlight_max(data):
    max_index = np.argmax(data, axis=1)
    for i, max_col in enumerate(max_index):
        data[i, max_col] = data[i, max_col]
    return data

# Plot the heatmap for throughput
heatmap_throughput = metric_df.pivot(
    index='latency', columns='scale', values='throughput')
ax = sns.heatmap(heatmap_throughput, ax=axes[0], cmap="YlGnBu", annot=True, fmt='.3g')
axes[0].set_title('Throughput vs. Latency and Scale')
annotate_extrema(heatmap_throughput.values, ax)

# Plot heatmap for total error (target deviation + osd)
metric_df['total_error'] = metric_df['avg_osd'] + metric_df['avg_target_error']
heatmap_error = metric_df.pivot(
    index='latency', columns='scale', values='total_error')
ax = sns.heatmap(heatmap_error, ax=axes[1], cmap="YlGnBu", annot=True, fmt='.3g')
axes[1].set_title('Total Error vs. Latency and Scale')
annotate_extrema(heatmap_error.values, ax, extrema_type='min')

# Plot heatmap for combined performance (movement speed - total error)
heatmap_combo = metric_df.pivot(
    index='latency', columns='scale', values='weighted_performance')
ax = sns.heatmap(heatmap_combo, ax=axes[2], cmap="YlGnBu", annot=True, fmt='.3g')
axes[2].set_title('Combined Performance vs. Latency and Scale')
annotate_extrema(heatmap_combo.values, ax, extrema_type='max')

# plt.title("User A")
plt.tight_layout()
# plt.savefig(f"{data_folder}/heatmap_key_metrics.png")
plt.show()

In [ ]:
# Define the function, e.g., a sine wave function
def smooth_2d_function(x, y):
    return np.sin(np.sqrt(x**2 + y**2))

# Generate sample points
x = np.linspace(-5, 5, 10)
y = np.linspace(-5, 5, 10)
x, y = np.meshgrid(x, y)

# Apply the function to the sample points
z = smooth_2d_function(x, y)

# Create a DataFrame
df = pd.DataFrame({'x': x.ravel(), 'y': y.ravel(), 'z': z.ravel()})

X = df[['x', 'y']]
Y = df['z']

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, train_size=0.8)

z_pred, _ = GPRegression(X_train, Y_train, X)
z_pred = z_pred.reshape(x.shape)

# df["y_pred"] = Y_pred

# Plotting the function for visualization
fig, axes = plt.subplots(1, 2, figsize=(10, 8))

ax = axes[0].contourf(x, y, z, cmap='viridis')
axes[0].set_xlabel('x')
axes[0].set_ylabel('y')
axes[0].set_title('Smooth 2D Function')

ax = axes[1].contourf(x, y, z_pred, cmap='viridis')
axes[1].set_xlabel('x')
axes[1].set_ylabel('y')
axes[1].set_title('Predictions')

fig.colorbar(ax, label='Function Value')
